In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import glob
import os
from datetime import datetime
pd.options.display.float_format = '{:,.2f}'.format

# READING FILES

In [2]:
date_respuesta = '2021-11-02'
date_respuesta = pd.to_datetime(date_respuesta)

## ARCHIVO MT

In [3]:
name_header=['mt']

In [4]:
mt_file = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2021\11. NOVIEMBRE\MT*.csv', recursive=False)
mt_file

['C:\\Users\\Cristian.Aparicio\\Documents\\FAST PAYMENTS\\2021\\11. NOVIEMBRE\\MT942 - 2021.11.02.csv']

In [5]:
mt = pd.concat([pd.read_csv(f,names=name_header,dtype=str).assign(file_name=os.path.basename(f)) for f in mt_file])
mt.head()

,mt,file_name
0,:20:17CE6801612682BA,MT942 - 2021.11.02.csv
1,:21:20211103095206,MT942 - 2021.11.02.csv
2,:25:70047593261,MT942 - 2021.11.02.csv
3,:28C:,MT942 - 2021.11.02.csv
4,":34F:MXND0,01",MT942 - 2021.11.02.csv


In [6]:
mt.shape

(13230, 2)

In [7]:
split = mt["mt"].str.split(",",n=7,expand=True)
split.head()

,0,1
0,:20:17CE6801612682BA,None
1,:21:20211103095206,None
2,:25:70047593261,None
3,:28C:,None
4,:34F:MXND0,01


In [8]:
mt["header_1"] = split[0] 
mt["header_2"] = split[1]

In [9]:
split_2 = mt['header_2'].str.split("//",n=7,expand=True).astype(str).replace(' ', '',regex=True)
split_2.head()

,0,1
0,None,None
1,None,None
2,None,None
3,None,None
4,01,None


In [10]:
mt['info_report_id'] = split_2[0]
mt['clave_rastreo'] = split_2[1]

In [11]:
mt_final = mt[(mt['clave_rastreo'] != 'None')]
mt_final['report_id'] = mt_final['info_report_id'].str[-9:].replace(' ', '',regex=True)
mt_final.head()

C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,mt,file_name,header_1,header_2,info_report_id,clave_rastreo,report_id
7,":61:2111031103DN30,28NMSCNLV2CNMPB//17CE680161...",MT942 - 2021.11.02.csv,:61:2111031103DN30,28NMSCNLV2CNMPB//17CE6801612682BA,28NMSCNLV2CNMPB,17CE6801612682BA,NLV2CNMPB
9,":61:2111031103DN440,69NMSCPDHZ9SYVT//17CE68015...",MT942 - 2021.11.02.csv,:61:2111031103DN440,69NMSCPDHZ9SYVT//17CE680158A1A66D,69NMSCPDHZ9SYVT,17CE680158A1A66D,PDHZ9SYVT
11,":61:2111031103DN67,07NMSCY29BPVQ6S//17CE680159...",MT942 - 2021.11.02.csv,:61:2111031103DN67,07NMSCY29BPVQ6S//17CE6801591A8E6D,07NMSCY29BPVQ6S,17CE6801591A8E6D,Y29BPVQ6S
13,":61:2111031103DN191,65NMSC375MN9N2J//17CE68015...",MT942 - 2021.11.02.csv,:61:2111031103DN191,65NMSC375MN9N2J//17CE68015AE17241,65NMSC375MN9N2J,17CE68015AE17241,375MN9N2J
15,":61:2111031103DN316,22NMSC9J2Y98ZKR//17CE68015...",MT942 - 2021.11.02.csv,:61:2111031103DN316,22NMSC9J2Y98ZKR//17CE68015B44D2C6,22NMSC9J2Y98ZKR,17CE68015B44D2C6,9J2Y98ZKR


In [12]:
mt_final.shape

(6605, 7)

## RESPUESTAS FP

In [13]:
rs = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2021\11. NOVIEMBRE\Respuesta-FP-CITI*.xlsx', recursive=False)
rs

['C:\\Users\\Cristian.Aparicio\\Documents\\FAST PAYMENTS\\2021\\11. NOVIEMBRE\\Respuesta-FP-CITI-1 2021-11-02.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\FAST PAYMENTS\\2021\\11. NOVIEMBRE\\Respuesta-FP-CITI-2 2021-11-02.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\FAST PAYMENTS\\2021\\11. NOVIEMBRE\\Respuesta-FP-CITI-3 2021-11-02.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\FAST PAYMENTS\\2021\\11. NOVIEMBRE\\Respuesta-FP-CITI-4 2021-11-02.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\FAST PAYMENTS\\2021\\11. NOVIEMBRE\\Respuesta-FP-CITI-5 2021-11-02.xlsx',
 'C:\\Users\\Cristian.Aparicio\\Documents\\FAST PAYMENTS\\2021\\11. NOVIEMBRE\\Respuesta-FP-CITI-6 2021-11-02.xlsx']

In [14]:
rs_concat = pd.concat([pd.read_excel(f,sheet_name='Sheet1',converters={'dsp_reference_id':str,'bank_account_number':str
                                                                     ,'disbursement_tracking_key':str,'report_id':str})
                       .assign(file_name=os.path.basename(f)) for f in rs])
rs_concat.loc[rs_concat['file_name'].str.contains('CITI-1'), 'Time_rs'] = '09:00'
rs_concat.loc[rs_concat['file_name'].str.contains('CITI-2'), 'Time_rs'] = '11:00'
rs_concat.loc[rs_concat['file_name'].str.contains('CITI-3'), 'Time_rs'] = '14:00'
rs_concat.loc[rs_concat['file_name'].str.contains('CITI-4'), 'Time_rs'] = '16:00'
rs_concat.loc[rs_concat['file_name'].str.contains('CITI-5'), 'Time_rs'] = '18:00'
rs_concat.loc[rs_concat['file_name'].str.contains('CITI-6'), 'Time_rs'] = '20:00'
rs_concat['INDEX'] = (rs_concat['report_id'].astype(str).replace(' ', '',regex=True)+'_'
                       +rs_concat['Time_rs'].astype(str).replace(' ', '',regex=True))
rs_concat.head()

,report_id,status,report_date_local,payment_date_local,amount,disbursement_batch_id,dsp_reference_id,entry_number,disbursement_id,bank_account_number,...,bank_account_owner_name,execution_date,application_date,number_movement,created_at,updated_at,error_description,file_name,Time_rs,INDEX
0,X29GNRGB2,PAID,2021-11-02,2021-11-02,498.28,6d44de0a-8242-456f-8349-e0ba14923fac,0900001,0,b968db69-29df-4270-bba3-4646cd29a640,659802020000256205,...,Jose Salvador Valenzuela Mares,2021-11-02,2021-11-02,0,2021-11-02,2021-11-02,NaN,Respuesta-FP-CITI-1 2021-11-02.xlsx,09:00,X29GNRGB2_09:00
1,NS53SZMZ3,PAID,2021-11-02,2021-11-02,114.99,6d44de0a-8242-456f-8349-e0ba14923fac,0900002,1,1ac17df2-6e0c-4bb9-8af2-a26282399bc0,659802020000281607,...,Arnoldo Toledano Gutierrez,2021-11-02,2021-11-02,0,2021-11-02,2021-11-02,NaN,Respuesta-FP-CITI-1 2021-11-02.xlsx,09:00,NS53SZMZ3_09:00
2,FCVF26YWR,PAID,2021-11-02,2021-11-02,216.56,6d44de0a-8242-456f-8349-e0ba14923fac,0900003,2,0a6a6599-1c42-49d9-b670-e2d178ecf89a,659802020000276573,...,Felix Ilich Ortiz Rangel,2021-11-02,2021-11-02,0,2021-11-02,2021-11-02,NaN,Respuesta-FP-CITI-1 2021-11-02.xlsx,09:00,FCVF26YWR_09:00
3,NLSSFXG4Z,PAID,2021-11-02,2021-11-02,"1,281.17",6d44de0a-8242-456f-8349-e0ba14923fac,0900004,3,458cdc2a-7ee7-465b-b4d6-718878f7fb9a,659802020000162328,...,Jesus Daniel Icedo Paredes,2021-11-02,2021-11-02,0,2021-11-02,2021-11-02,NaN,Respuesta-FP-CITI-1 2021-11-02.xlsx,09:00,NLSSFXG4Z_09:00
4,GY5X3K97N,PAID,2021-11-02,2021-11-02,19.97,6d44de0a-8242-456f-8349-e0ba14923fac,0900005,4,3bc9f008-2e96-44df-aaef-98df4ffddecd,659802020000300038,...,Miguel Hernandez Ramirez,2021-11-02,2021-11-02,0,2021-11-02,2021-11-02,NaN,Respuesta-FP-CITI-1 2021-11-02.xlsx,09:00,GY5X3K97N_09:00


In [15]:
rs_concat.shape

(5573, 21)

In [16]:
duplicates = rs_concat[rs_concat.duplicated(['report_id'] , keep=False)]
duplicates.head()

,report_id,status,report_date_local,payment_date_local,amount,disbursement_batch_id,dsp_reference_id,entry_number,disbursement_id,bank_account_number,...,bank_account_owner_name,execution_date,application_date,number_movement,created_at,updated_at,error_description,file_name,Time_rs,INDEX


In [17]:
duplicates.shape

(0, 21)

## QUERY PAYMENTS

In [18]:
query = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2021\11. NOVIEMBRE\FP-CITI 2021-11-02.csv')
query['batch_created_date'] = pd.to_datetime(query['batch_created_date'])
query.set_index('batch_created_date',inplace=True)
query.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,status,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,swap
batch_created_date,,,,,,,,,,,,
2021-11-02 09:46:10,002614a8-92f6-4d04-89c8-bc831434361d,VAMA System Solutions,uberchato1997@gmail.com,X29GNRGB2,2021-11-02 00:00:00,498.28,PAID,"Opciones Empresariales del Noreste, S.A. de C....",2021-11-02 09:52:17,Jose Salvador Valenzuela Mares,'659802020000256205,"Jose Salvador Valenzuela Mares,,498.28,6598020..."
2021-11-02 18:26:44,003e3797-50f2-45d4-94f9-c211dc599854,El Pollo TOM,rpalomares.celulares@gmail.com,HQHFDVJ8B,2021-11-02 00:00:00,57.49,PAID,"Opciones Empresariales del Noreste, S.A. de C....",2021-11-02 18:34:30,Ricardo Enrique Palomares García,'659802020000047838,"Ricardo Enrique Palomares García,,57.49,659802..."
2021-11-02 20:46:06,0042f78c-1d9d-4db4-a21d-ee3b9130d44f,Alessa salon spa,alejandra.ponce.sanchez@gmail.com,VXXVD8TRN,2021-11-02 00:00:00,"3,832.96",PAID,"Opciones Empresariales del Noreste, S.A. de C....",2021-11-02 20:54:39,Alejandra Ponce Sanchez,'659802020000137382,"Alejandra Ponce Sanchez,,3832.96,6598020200001..."
2021-11-02 20:46:06,0044619f-0662-48cd-9718-cdee897d5f7a,La Jaula,lajaulasoccer5@gmail.com,FPMSWD6GT,2021-11-02 00:00:00,153.32,PAID,"Opciones Empresariales del Noreste, S.A. de C....",2021-11-02 20:54:39,Leonardo Rojas Pacheco,'659802020000515069,"Leonardo Rojas Pacheco,,153.32,659802020000515..."
2021-11-02 14:06:01,00731f44-d2f0-4a8d-b87d-6db15b720b00,NMO,nathaliemorgado7@gmail.com,TB84BZLWP,2021-11-02 00:00:00,"1,149.89",PAID,"Opciones Empresariales del Noreste, S.A. de C....",2021-11-02 14:12:07,Nathalie Morgado Ortega,'659802020000053819,"Nathalie Morgado Ortega,,1149.89,6598020200000..."


In [19]:
query.shape

(5573, 12)

In [20]:
hour1 = query.between_time('08:00','10:59')
hour1['Time'] = '09:00'
hour2 = query.between_time('11:00','13:59')
hour2['Time'] = '11:00'
hour3 = query.between_time('14:00','15:59')
hour3['Time'] = '14:00'
hour4 = query.between_time('16:00','17:59')
hour4['Time'] = '16:00'
hour5 = query.between_time('18:00','19:59')
hour5['Time'] = '18:00'
hour6 = query.between_time('20:00','23:59')
hour6['Time'] = '20:00'

C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [21]:
disp1 = pd.concat([hour1,hour2,hour3,hour4,hour5,hour6])
disp1['INDEX'] = (disp1['report_id'].astype(str).replace(' ', '',regex=True)+'_'
                       +disp1['Time'].astype(str).replace(' ', '',regex=True))
disp1.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,status,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,swap,Time,INDEX
batch_created_date,,,,,,,,,,,,,,
2021-11-02 09:46:10,002614a8-92f6-4d04-89c8-bc831434361d,VAMA System Solutions,uberchato1997@gmail.com,X29GNRGB2,2021-11-02 00:00:00,498.28,PAID,"Opciones Empresariales del Noreste, S.A. de C....",2021-11-02 09:52:17,Jose Salvador Valenzuela Mares,'659802020000256205,"Jose Salvador Valenzuela Mares,,498.28,6598020...",09:00,X29GNRGB2_09:00
2021-11-02 09:46:10,00ae274c-a015-4d09-ae9e-ab557f4c156c,Cremeria Los Angeles,arnold0019@hotmail.com,NS53SZMZ3,2021-11-02 00:00:00,114.99,PAID,"Opciones Empresariales del Noreste, S.A. de C....",2021-11-02 09:52:23,Arnoldo Toledano Gutierrez,'659802020000281607,"Arnoldo Toledano Gutierrez,,114.99,65980202000...",09:00,NS53SZMZ3_09:00
2021-11-02 09:46:10,00d0dc90-b79a-4c33-9100-66ba255c3a46,PITBOSSES VEGAN BBQ,thephytophagous@gmail.com,FCVF26YWR,2021-11-02 00:00:00,216.56,PAID,"Opciones Empresariales del Noreste, S.A. de C....",2021-11-02 09:52:22,Felix Ilich Ortiz Rangel,'659802020000276573,"Felix Ilich Ortiz Rangel,,216.56,6598020200002...",09:00,FCVF26YWR_09:00
2021-11-02 09:46:10,00d6a6e0-3374-4a17-910d-586e503543ea,Green-Go Mini Market,jesusicedo3@gmail.com,NLSSFXG4Z,2021-11-02 00:00:00,"1,281.17",PAID,"Opciones Empresariales del Noreste, S.A. de C....",2021-11-02 09:52:22,Jesus Daniel Icedo Paredes,'659802020000162328,"Jesus Daniel Icedo Paredes,,1281.17,6598020200...",09:00,NLSSFXG4Z_09:00
2021-11-02 09:46:10,01054eec-e90e-42a8-9471-3fb1711cbbc1,Abarrotes La Perla De Magon,laperlademagon2021@hotmail.com,GY5X3K97N,2021-11-02 00:00:00,19.97,PAID,"Opciones Empresariales del Noreste, S.A. de C....",2021-11-02 09:52:22,Miguel Hernandez Ramirez,'659802020000300038,"Miguel Hernandez Ramirez,,19.97,65980202000030...",09:00,GY5X3K97N_09:00


In [22]:
disp1.shape

(5573, 14)

## BIN

In [23]:
bin1 = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\DISPERSIONES\Catalogo_Bancos.csv'
                  ,dtype={'BIN':str,'Clave de la institución':str,'Nombre de la institución':str})


In [24]:
bin1.head()

,BIN,Clave de la institución,Nombre de la institución
0,138,40138,ABC CAPITAL
1,102,40102,ACCENDO BANCO
2,133,40133,ACTINVER
3,062,40062,AFIRME
4,638,90638,AKALA


In [25]:
bin1.shape

(88, 3)

# ARCHIVO RESPUESTA

In [26]:
rs_concat['Date_respuesta'] = date_respuesta
rs_concat.rename(columns ={'bank_account_number':'CLABE'}, inplace=True)
rs_concat['BIN'] = rs_concat['CLABE'].str[:3].replace(' ', '',regex=True)
rs_concat.head()

,report_id,status,report_date_local,payment_date_local,amount,disbursement_batch_id,dsp_reference_id,entry_number,disbursement_id,CLABE,...,application_date,number_movement,created_at,updated_at,error_description,file_name,Time_rs,INDEX,Date_respuesta,BIN
0,X29GNRGB2,PAID,2021-11-02,2021-11-02,498.28,6d44de0a-8242-456f-8349-e0ba14923fac,0900001,0,b968db69-29df-4270-bba3-4646cd29a640,659802020000256205,...,2021-11-02,0,2021-11-02,2021-11-02,NaN,Respuesta-FP-CITI-1 2021-11-02.xlsx,09:00,X29GNRGB2_09:00,2021-11-02,659
1,NS53SZMZ3,PAID,2021-11-02,2021-11-02,114.99,6d44de0a-8242-456f-8349-e0ba14923fac,0900002,1,1ac17df2-6e0c-4bb9-8af2-a26282399bc0,659802020000281607,...,2021-11-02,0,2021-11-02,2021-11-02,NaN,Respuesta-FP-CITI-1 2021-11-02.xlsx,09:00,NS53SZMZ3_09:00,2021-11-02,659
2,FCVF26YWR,PAID,2021-11-02,2021-11-02,216.56,6d44de0a-8242-456f-8349-e0ba14923fac,0900003,2,0a6a6599-1c42-49d9-b670-e2d178ecf89a,659802020000276573,...,2021-11-02,0,2021-11-02,2021-11-02,NaN,Respuesta-FP-CITI-1 2021-11-02.xlsx,09:00,FCVF26YWR_09:00,2021-11-02,659
3,NLSSFXG4Z,PAID,2021-11-02,2021-11-02,"1,281.17",6d44de0a-8242-456f-8349-e0ba14923fac,0900004,3,458cdc2a-7ee7-465b-b4d6-718878f7fb9a,659802020000162328,...,2021-11-02,0,2021-11-02,2021-11-02,NaN,Respuesta-FP-CITI-1 2021-11-02.xlsx,09:00,NLSSFXG4Z_09:00,2021-11-02,659
4,GY5X3K97N,PAID,2021-11-02,2021-11-02,19.97,6d44de0a-8242-456f-8349-e0ba14923fac,0900005,4,3bc9f008-2e96-44df-aaef-98df4ffddecd,659802020000300038,...,2021-11-02,0,2021-11-02,2021-11-02,NaN,Respuesta-FP-CITI-1 2021-11-02.xlsx,09:00,GY5X3K97N_09:00,2021-11-02,659


In [27]:
rs_concat.shape

(5573, 23)

In [28]:
rs_concat_2 = rs_concat.merge(bin1[['BIN','Clave de la institución','Nombre de la institución']] ,on='BIN', how='left')
rs_concat_2['Banco emisor'] = '40002'
rs_concat_2['Banco_emisor'] = 'BANAMEX'
rs_concat_2.head()

,report_id,status,report_date_local,payment_date_local,amount,disbursement_batch_id,dsp_reference_id,entry_number,disbursement_id,CLABE,...,error_description,file_name,Time_rs,INDEX,Date_respuesta,BIN,Clave de la institución,Nombre de la institución,Banco emisor,Banco_emisor
0,X29GNRGB2,PAID,2021-11-02,2021-11-02,498.28,6d44de0a-8242-456f-8349-e0ba14923fac,0900001,0,b968db69-29df-4270-bba3-4646cd29a640,659802020000256205,...,NaN,Respuesta-FP-CITI-1 2021-11-02.xlsx,09:00,X29GNRGB2_09:00,2021-11-02,659,90659,ASP INTEGRA OPC,40002,BANAMEX
1,NS53SZMZ3,PAID,2021-11-02,2021-11-02,114.99,6d44de0a-8242-456f-8349-e0ba14923fac,0900002,1,1ac17df2-6e0c-4bb9-8af2-a26282399bc0,659802020000281607,...,NaN,Respuesta-FP-CITI-1 2021-11-02.xlsx,09:00,NS53SZMZ3_09:00,2021-11-02,659,90659,ASP INTEGRA OPC,40002,BANAMEX
2,FCVF26YWR,PAID,2021-11-02,2021-11-02,216.56,6d44de0a-8242-456f-8349-e0ba14923fac,0900003,2,0a6a6599-1c42-49d9-b670-e2d178ecf89a,659802020000276573,...,NaN,Respuesta-FP-CITI-1 2021-11-02.xlsx,09:00,FCVF26YWR_09:00,2021-11-02,659,90659,ASP INTEGRA OPC,40002,BANAMEX
3,NLSSFXG4Z,PAID,2021-11-02,2021-11-02,"1,281.17",6d44de0a-8242-456f-8349-e0ba14923fac,0900004,3,458cdc2a-7ee7-465b-b4d6-718878f7fb9a,659802020000162328,...,NaN,Respuesta-FP-CITI-1 2021-11-02.xlsx,09:00,NLSSFXG4Z_09:00,2021-11-02,659,90659,ASP INTEGRA OPC,40002,BANAMEX
4,GY5X3K97N,PAID,2021-11-02,2021-11-02,19.97,6d44de0a-8242-456f-8349-e0ba14923fac,0900005,4,3bc9f008-2e96-44df-aaef-98df4ffddecd,659802020000300038,...,NaN,Respuesta-FP-CITI-1 2021-11-02.xlsx,09:00,GY5X3K97N_09:00,2021-11-02,659,90659,ASP INTEGRA OPC,40002,BANAMEX


In [29]:
rs_concat_2.shape

(5573, 27)

In [30]:
rs_concat_3 = rs_concat_2.merge(mt_final[['report_id','clave_rastreo']] ,on='report_id', how='left')
rs_concat_3['LINEA CEP'] = (rs_concat_3['Date_respuesta'].astype(str).replace(' ', '',regex=True)+','
                            +rs_concat_3['clave_rastreo'].astype(str).replace(' ', '',regex=True)+','
                            +rs_concat_3['Banco emisor'].astype(str).replace(' ', '',regex=True)+','
                            +rs_concat_3['Clave de la institución'].astype(str).replace(' ', '',regex=True)+','
                            +rs_concat_3['CLABE'].astype(str).replace(' ', '',regex=True)+','
                            +rs_concat_3['amount'].astype(str).replace(' ', '',regex=True))
rs_concat_3.head()

,report_id,status,report_date_local,payment_date_local,amount,disbursement_batch_id,dsp_reference_id,entry_number,disbursement_id,CLABE,...,Time_rs,INDEX,Date_respuesta,BIN,Clave de la institución,Nombre de la institución,Banco emisor,Banco_emisor,clave_rastreo,LINEA CEP
0,X29GNRGB2,PAID,2021-11-02,2021-11-02,498.28,6d44de0a-8242-456f-8349-e0ba14923fac,0900001,0,b968db69-29df-4270-bba3-4646cd29a640,659802020000256205,...,09:00,X29GNRGB2_09:00,2021-11-02,659,90659,ASP INTEGRA OPC,40002,BANAMEX,17CE1596C9564A3F,"2021-11-02,17CE1596C9564A3F,40002,90659,659802..."
1,NS53SZMZ3,PAID,2021-11-02,2021-11-02,114.99,6d44de0a-8242-456f-8349-e0ba14923fac,0900002,1,1ac17df2-6e0c-4bb9-8af2-a26282399bc0,659802020000281607,...,09:00,NS53SZMZ3_09:00,2021-11-02,659,90659,ASP INTEGRA OPC,40002,BANAMEX,17CE159798ECFD4C,"2021-11-02,17CE159798ECFD4C,40002,90659,659802..."
2,FCVF26YWR,PAID,2021-11-02,2021-11-02,216.56,6d44de0a-8242-456f-8349-e0ba14923fac,0900003,2,0a6a6599-1c42-49d9-b670-e2d178ecf89a,659802020000276573,...,09:00,FCVF26YWR_09:00,2021-11-02,659,90659,ASP INTEGRA OPC,40002,BANAMEX,17CE159783C7AC76,"2021-11-02,17CE159783C7AC76,40002,90659,659802..."
3,NLSSFXG4Z,PAID,2021-11-02,2021-11-02,"1,281.17",6d44de0a-8242-456f-8349-e0ba14923fac,0900004,3,458cdc2a-7ee7-465b-b4d6-718878f7fb9a,659802020000162328,...,09:00,NLSSFXG4Z_09:00,2021-11-02,659,90659,ASP INTEGRA OPC,40002,BANAMEX,17CE159779CAA01C,"2021-11-02,17CE159779CAA01C,40002,90659,659802..."
4,GY5X3K97N,PAID,2021-11-02,2021-11-02,19.97,6d44de0a-8242-456f-8349-e0ba14923fac,0900005,4,3bc9f008-2e96-44df-aaef-98df4ffddecd,659802020000300038,...,09:00,GY5X3K97N_09:00,2021-11-02,659,90659,ASP INTEGRA OPC,40002,BANAMEX,17CE1597749F5CFF,"2021-11-02,17CE1597749F5CFF,40002,90659,659802..."


In [31]:
rs_concat_3.shape

(5573, 29)

# ARCHIVO DISPERSION

In [32]:
disp1.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,status,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,swap,Time,INDEX
batch_created_date,,,,,,,,,,,,,,
2021-11-02 09:46:10,002614a8-92f6-4d04-89c8-bc831434361d,VAMA System Solutions,uberchato1997@gmail.com,X29GNRGB2,2021-11-02 00:00:00,498.28,PAID,"Opciones Empresariales del Noreste, S.A. de C....",2021-11-02 09:52:17,Jose Salvador Valenzuela Mares,'659802020000256205,"Jose Salvador Valenzuela Mares,,498.28,6598020...",09:00,X29GNRGB2_09:00
2021-11-02 09:46:10,00ae274c-a015-4d09-ae9e-ab557f4c156c,Cremeria Los Angeles,arnold0019@hotmail.com,NS53SZMZ3,2021-11-02 00:00:00,114.99,PAID,"Opciones Empresariales del Noreste, S.A. de C....",2021-11-02 09:52:23,Arnoldo Toledano Gutierrez,'659802020000281607,"Arnoldo Toledano Gutierrez,,114.99,65980202000...",09:00,NS53SZMZ3_09:00
2021-11-02 09:46:10,00d0dc90-b79a-4c33-9100-66ba255c3a46,PITBOSSES VEGAN BBQ,thephytophagous@gmail.com,FCVF26YWR,2021-11-02 00:00:00,216.56,PAID,"Opciones Empresariales del Noreste, S.A. de C....",2021-11-02 09:52:22,Felix Ilich Ortiz Rangel,'659802020000276573,"Felix Ilich Ortiz Rangel,,216.56,6598020200002...",09:00,FCVF26YWR_09:00
2021-11-02 09:46:10,00d6a6e0-3374-4a17-910d-586e503543ea,Green-Go Mini Market,jesusicedo3@gmail.com,NLSSFXG4Z,2021-11-02 00:00:00,"1,281.17",PAID,"Opciones Empresariales del Noreste, S.A. de C....",2021-11-02 09:52:22,Jesus Daniel Icedo Paredes,'659802020000162328,"Jesus Daniel Icedo Paredes,,1281.17,6598020200...",09:00,NLSSFXG4Z_09:00
2021-11-02 09:46:10,01054eec-e90e-42a8-9471-3fb1711cbbc1,Abarrotes La Perla De Magon,laperlademagon2021@hotmail.com,GY5X3K97N,2021-11-02 00:00:00,19.97,PAID,"Opciones Empresariales del Noreste, S.A. de C....",2021-11-02 09:52:22,Miguel Hernandez Ramirez,'659802020000300038,"Miguel Hernandez Ramirez,,19.97,65980202000030...",09:00,GY5X3K97N_09:00


In [33]:
disp1['report_date_local'] = pd.to_datetime(disp1['report_date_local'])

In [34]:
split = disp1["swap"].str.split(",",n=7,expand=True)
split.head()

,0,1,2,3,4
batch_created_date,,,,,
2021-11-02 09:46:10,Jose Salvador Valenzuela Mares,,498.28,659802020000256205,CLIP-X29GNRGB2
2021-11-02 09:46:10,Arnoldo Toledano Gutierrez,,114.99,659802020000281607,CLIP-NS53SZMZ3
2021-11-02 09:46:10,Felix Ilich Ortiz Rangel,,216.56,659802020000276573,CLIP-FCVF26YWR
2021-11-02 09:46:10,Jesus Daniel Icedo Paredes,,1281.17,659802020000162328,CLIP-NLSSFXG4Z
2021-11-02 09:46:10,Miguel Hernandez Ramirez,,19.97,659802020000300038,CLIP-GY5X3K97N


In [35]:
disp1["swap"] = split[0] 
disp1["CLABE"] = split[3]

In [36]:
disp1.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,status,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,swap,Time,INDEX,CLABE
batch_created_date,,,,,,,,,,,,,,,
2021-11-02 09:46:10,002614a8-92f6-4d04-89c8-bc831434361d,VAMA System Solutions,uberchato1997@gmail.com,X29GNRGB2,2021-11-02,498.28,PAID,"Opciones Empresariales del Noreste, S.A. de C....",2021-11-02 09:52:17,Jose Salvador Valenzuela Mares,'659802020000256205,Jose Salvador Valenzuela Mares,09:00,X29GNRGB2_09:00,659802020000256205
2021-11-02 09:46:10,00ae274c-a015-4d09-ae9e-ab557f4c156c,Cremeria Los Angeles,arnold0019@hotmail.com,NS53SZMZ3,2021-11-02,114.99,PAID,"Opciones Empresariales del Noreste, S.A. de C....",2021-11-02 09:52:23,Arnoldo Toledano Gutierrez,'659802020000281607,Arnoldo Toledano Gutierrez,09:00,NS53SZMZ3_09:00,659802020000281607
2021-11-02 09:46:10,00d0dc90-b79a-4c33-9100-66ba255c3a46,PITBOSSES VEGAN BBQ,thephytophagous@gmail.com,FCVF26YWR,2021-11-02,216.56,PAID,"Opciones Empresariales del Noreste, S.A. de C....",2021-11-02 09:52:22,Felix Ilich Ortiz Rangel,'659802020000276573,Felix Ilich Ortiz Rangel,09:00,FCVF26YWR_09:00,659802020000276573
2021-11-02 09:46:10,00d6a6e0-3374-4a17-910d-586e503543ea,Green-Go Mini Market,jesusicedo3@gmail.com,NLSSFXG4Z,2021-11-02,"1,281.17",PAID,"Opciones Empresariales del Noreste, S.A. de C....",2021-11-02 09:52:22,Jesus Daniel Icedo Paredes,'659802020000162328,Jesus Daniel Icedo Paredes,09:00,NLSSFXG4Z_09:00,659802020000162328
2021-11-02 09:46:10,01054eec-e90e-42a8-9471-3fb1711cbbc1,Abarrotes La Perla De Magon,laperlademagon2021@hotmail.com,GY5X3K97N,2021-11-02,19.97,PAID,"Opciones Empresariales del Noreste, S.A. de C....",2021-11-02 09:52:22,Miguel Hernandez Ramirez,'659802020000300038,Miguel Hernandez Ramirez,09:00,GY5X3K97N_09:00,659802020000300038


In [37]:
disp1.loc[disp1['report_date_local'] == date_respuesta, 'tipo'] = 'Pago del día'
disp1.loc[disp1['report_date_local'] != date_respuesta, 'tipo'] = 'Reprogramación'
disp1.drop(['bank_name','CLABE','status'],axis=1,inplace=True)
disp1.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,payment_date_local,bank_account_owner_name,bank_account_number,swap,Time,INDEX,tipo
batch_created_date,,,,,,,,,,,,,
2021-11-02 09:46:10,002614a8-92f6-4d04-89c8-bc831434361d,VAMA System Solutions,uberchato1997@gmail.com,X29GNRGB2,2021-11-02,498.28,2021-11-02 09:52:17,Jose Salvador Valenzuela Mares,'659802020000256205,Jose Salvador Valenzuela Mares,09:00,X29GNRGB2_09:00,Pago del día
2021-11-02 09:46:10,00ae274c-a015-4d09-ae9e-ab557f4c156c,Cremeria Los Angeles,arnold0019@hotmail.com,NS53SZMZ3,2021-11-02,114.99,2021-11-02 09:52:23,Arnoldo Toledano Gutierrez,'659802020000281607,Arnoldo Toledano Gutierrez,09:00,NS53SZMZ3_09:00,Pago del día
2021-11-02 09:46:10,00d0dc90-b79a-4c33-9100-66ba255c3a46,PITBOSSES VEGAN BBQ,thephytophagous@gmail.com,FCVF26YWR,2021-11-02,216.56,2021-11-02 09:52:22,Felix Ilich Ortiz Rangel,'659802020000276573,Felix Ilich Ortiz Rangel,09:00,FCVF26YWR_09:00,Pago del día
2021-11-02 09:46:10,00d6a6e0-3374-4a17-910d-586e503543ea,Green-Go Mini Market,jesusicedo3@gmail.com,NLSSFXG4Z,2021-11-02,"1,281.17",2021-11-02 09:52:22,Jesus Daniel Icedo Paredes,'659802020000162328,Jesus Daniel Icedo Paredes,09:00,NLSSFXG4Z_09:00,Pago del día
2021-11-02 09:46:10,01054eec-e90e-42a8-9471-3fb1711cbbc1,Abarrotes La Perla De Magon,laperlademagon2021@hotmail.com,GY5X3K97N,2021-11-02,19.97,2021-11-02 09:52:22,Miguel Hernandez Ramirez,'659802020000300038,Miguel Hernandez Ramirez,09:00,GY5X3K97N_09:00,Pago del día


In [38]:
disp_2 = disp1.merge(rs_concat_3[['report_id','status','Nombre de la institución','CLABE','LINEA CEP','Time_rs','Banco_emisor']]
                     ,on='report_id', how='left')
disp_2.sort_values(by=['status'])
disp_2["Time"].fillna('NO TIME',inplace=True)
disp_2.loc[disp_2['Time'] == 'NO TIME', 'status'] = 'ERROR'
disp_2.loc[(disp_2['LINEA CEP'].str.contains('nan')) & (['Nombre de la institución'] == 'BANAMEX'), 'status'] = 'ERROR'
# disp_2.loc[pd.notnull(disp_2['error_description']), 'status'] = 'ERROR'
disp_2.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,payment_date_local,bank_account_owner_name,bank_account_number,swap,Time,INDEX,tipo,status,Nombre de la institución,CLABE,LINEA CEP,Time_rs,Banco_emisor
0,002614a8-92f6-4d04-89c8-bc831434361d,VAMA System Solutions,uberchato1997@gmail.com,X29GNRGB2,2021-11-02,498.28,2021-11-02 09:52:17,Jose Salvador Valenzuela Mares,'659802020000256205,Jose Salvador Valenzuela Mares,09:00,X29GNRGB2_09:00,Pago del día,PAID,ASP INTEGRA OPC,659802020000256205,"2021-11-02,17CE1596C9564A3F,40002,90659,659802...",09:00,BANAMEX
1,00ae274c-a015-4d09-ae9e-ab557f4c156c,Cremeria Los Angeles,arnold0019@hotmail.com,NS53SZMZ3,2021-11-02,114.99,2021-11-02 09:52:23,Arnoldo Toledano Gutierrez,'659802020000281607,Arnoldo Toledano Gutierrez,09:00,NS53SZMZ3_09:00,Pago del día,PAID,ASP INTEGRA OPC,659802020000281607,"2021-11-02,17CE159798ECFD4C,40002,90659,659802...",09:00,BANAMEX
2,00d0dc90-b79a-4c33-9100-66ba255c3a46,PITBOSSES VEGAN BBQ,thephytophagous@gmail.com,FCVF26YWR,2021-11-02,216.56,2021-11-02 09:52:22,Felix Ilich Ortiz Rangel,'659802020000276573,Felix Ilich Ortiz Rangel,09:00,FCVF26YWR_09:00,Pago del día,PAID,ASP INTEGRA OPC,659802020000276573,"2021-11-02,17CE159783C7AC76,40002,90659,659802...",09:00,BANAMEX
3,00d6a6e0-3374-4a17-910d-586e503543ea,Green-Go Mini Market,jesusicedo3@gmail.com,NLSSFXG4Z,2021-11-02,"1,281.17",2021-11-02 09:52:22,Jesus Daniel Icedo Paredes,'659802020000162328,Jesus Daniel Icedo Paredes,09:00,NLSSFXG4Z_09:00,Pago del día,PAID,ASP INTEGRA OPC,659802020000162328,"2021-11-02,17CE159779CAA01C,40002,90659,659802...",09:00,BANAMEX
4,01054eec-e90e-42a8-9471-3fb1711cbbc1,Abarrotes La Perla De Magon,laperlademagon2021@hotmail.com,GY5X3K97N,2021-11-02,19.97,2021-11-02 09:52:22,Miguel Hernandez Ramirez,'659802020000300038,Miguel Hernandez Ramirez,09:00,GY5X3K97N_09:00,Pago del día,PAID,ASP INTEGRA OPC,659802020000300038,"2021-11-02,17CE1597749F5CFF,40002,90659,659802...",09:00,BANAMEX


In [39]:
disp_2.loc[disp_2['Nombre de la institución'] == 'BANAMEX', 'LINEA CEP'] = 'COMPROBANTE MISMO BANCO BANAMEX'
disp_2.loc[disp_2['status'] == 'ERROR', 'LINEA CEP'] = ''

In [40]:
for col in disp_2.columns: 
    print(col) 

merchant_id
merchant_name
user_email
report_id
report_date_local
amount
payment_date_local
bank_account_owner_name
bank_account_number
swap
Time
INDEX
tipo
status
Nombre de la institución
CLABE
LINEA CEP
Time_rs
Banco_emisor


In [41]:
disp_final = disp_2[['merchant_id','merchant_name','user_email','report_id','report_date_local','amount','tipo'
                    ,'Nombre de la institución','swap','CLABE','status','LINEA CEP','Time','Banco_emisor']]
disp_final.rename(columns ={'Nombre de la institución':'bank','amount':'total_due','LINEA CEP':'CEP'}, inplace=True)
disp_final.sort_values(by=['status','Time'], ascending=True, inplace=True)
disp_final.drop_duplicates(subset =['report_id','Time'], keep ='first', inplace=True)
disp_final.head()

C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,merchant_id,merchant_name,user_email,report_id,report_date_local,total_due,tipo,bank,swap,CLABE,status,CEP,Time,Banco_emisor
0,002614a8-92f6-4d04-89c8-bc831434361d,VAMA System Solutions,uberchato1997@gmail.com,X29GNRGB2,2021-11-02,498.28,Pago del día,ASP INTEGRA OPC,Jose Salvador Valenzuela Mares,659802020000256205,PAID,"2021-11-02,17CE1596C9564A3F,40002,90659,659802...",09:00,BANAMEX
1,00ae274c-a015-4d09-ae9e-ab557f4c156c,Cremeria Los Angeles,arnold0019@hotmail.com,NS53SZMZ3,2021-11-02,114.99,Pago del día,ASP INTEGRA OPC,Arnoldo Toledano Gutierrez,659802020000281607,PAID,"2021-11-02,17CE159798ECFD4C,40002,90659,659802...",09:00,BANAMEX
2,00d0dc90-b79a-4c33-9100-66ba255c3a46,PITBOSSES VEGAN BBQ,thephytophagous@gmail.com,FCVF26YWR,2021-11-02,216.56,Pago del día,ASP INTEGRA OPC,Felix Ilich Ortiz Rangel,659802020000276573,PAID,"2021-11-02,17CE159783C7AC76,40002,90659,659802...",09:00,BANAMEX
3,00d6a6e0-3374-4a17-910d-586e503543ea,Green-Go Mini Market,jesusicedo3@gmail.com,NLSSFXG4Z,2021-11-02,"1,281.17",Pago del día,ASP INTEGRA OPC,Jesus Daniel Icedo Paredes,659802020000162328,PAID,"2021-11-02,17CE159779CAA01C,40002,90659,659802...",09:00,BANAMEX
4,01054eec-e90e-42a8-9471-3fb1711cbbc1,Abarrotes La Perla De Magon,laperlademagon2021@hotmail.com,GY5X3K97N,2021-11-02,19.97,Pago del día,ASP INTEGRA OPC,Miguel Hernandez Ramirez,659802020000300038,PAID,"2021-11-02,17CE1597749F5CFF,40002,90659,659802...",09:00,BANAMEX


In [42]:
disp_final.shape

(5573, 14)

# ERROR

In [43]:
error = disp_final[(disp_final['status'] != 'PAID')]
error.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,total_due,tipo,bank,swap,CLABE,status,CEP,Time,Banco_emisor


In [44]:
error.shape

(0, 14)

In [45]:
error.drop(['swap','CLABE','status','CEP','Banco_emisor'],axis=1,inplace=True)

C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [46]:
error_final = error
error_final.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,total_due,tipo,bank,Time


In [ ]:
error.loc[error['bank'] == 'BANAMEX', 'Check'] = 'SPEI DEVUELTO MISMO BANCO'
error.loc[error['bank'] != 'BANAMEX', 'Check'] = 'SPEI DEVUELTO'

In [ ]:
error.shape

In [ ]:
error_final = error.merge(rs_concat_2[['report_id','error_description']],on='report_id',how='left')
error_final.head()

In [ ]:
error_final.shape

# ARCHIVOS FINALES

### ARCHIVO DISPERSION

In [47]:
writer = pd.ExcelWriter(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2021\11. NOVIEMBRE\ARCHIVO DISPERSIONES FP\CITI\Dispersión-FP-CITI 2021-11-02.xlsx', engine='xlsxwriter', options={'encoding':'utf-8'})
disp_final.to_excel(writer, sheet_name='FAST PAYMENTS',index=False)
error_final.to_excel(writer, sheet_name='ERROR',index=False)
writer.save()
writer.close()

### ARCHIVO DISPERSIÓN - TODO POR CITI

In [48]:
writer = pd.ExcelWriter(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2021\11. NOVIEMBRE\ARCHIVO DISPERSIONES FP\Dispersión-FP 2021-11-02.xlsx', engine='xlsxwriter', options={'encoding':'utf-8'})
disp_final.to_excel(writer, sheet_name='FAST PAYMENTS',index=False)
error_final.to_excel(writer, sheet_name='ERROR',index=False)
writer.save()
writer.close()

### ARCHIVO RESPUESTA 

In [49]:
rs_concat.to_csv(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2021\11. NOVIEMBRE\ARCHIVO RESPUESTAS FINALES\Respuesta-FP-CITI 2021-11-02.csv',index=False)

In [ ]:
# book = load_workbook('Dispersión-FP 2020-07-20.xlsx')
# writer = pd.ExcelWriter('Dispersión-FP 2020-07-20.xlsx', engine='openpyxl', options={'encoding':'utf-8'})
# writer.book = book
# match_final.to_excel(writer, sheet_name='FP-2 2020-07-20', index=False)
# writer.save()
# writer.close()